In [291]:
import pandas as pd
import ijson
from urllib.request import urlopen
import json
import os, sys
from dotenv import load_dotenv
import psycopg2 as pg
# TODO this all in updater file, update only all two days

In [292]:
# load environment variables 
load_dotenv()

# get environment variables
DB_CON_STRING = os.getenv('DB_CON_STRING', None)

# build up database connection
try:
    connection = pg.connect(DB_CON_STRING)
except Exception as e:
    printerr(e)

cur = connection.cursor()

In [98]:
json_url = 'https://dataex.ohchr.org/uhri/export-results/export-full-en.json'

In [294]:
f = urlopen(json_url)

In [295]:
objects = ijson.items(f, 'item')

In [296]:
observations = (o for o in objects if o['AnnotationType'] == '- Concerns/Observations')

In [297]:
with open('/home/me/projects/worldcrisismap/crisis_database/mappers/uhri_country_mapping.json', 'r') as f:  #TODO PATH RELATIONAL 
    country_dict = json.load(f)

In [ ]:
cur.execute('''
TRUNCATE TABLE human;
TRUNCATE TABLE human_text;
''')
connection.commit()

In [309]:
doc_data = {}  # COMMENT THIS

for o in observations:
    doc_id = o['DocumentId']
    doc = data.get(doc_id)

    if doc == None:
        doc_data[doc_id] = doc = {
            'date': o['PublicationDate'],
            'countries': set(o['Countries']),
        }

        db_cmd = 'INSERT INTO human_text(id, text) VALUES (%(id)s, %(text)s)'
    else:
        db_cmd = 'UPDATE human_text SET text = CONCAT(text, %(text)s) WHERE id = %(id)s;'

    try:
        cur.execute(db_cmd, {
                        'id': doc_id, 
                        'text': '<br>' + o['Text']
                    })
        connection.commit()
    except:
        connection.rollback()

    doc['countries'].update(o['Countries'])

In [9]:
# loop through each single document TODO
for doc_id in doc_data:
    # these information are needed (and loaded into database 'human')
    # - date
    # - country name
    # - geojson paths (from country iso3s)
    # - lat / lon
    # - title
    # - text

    try:
        # all unique country codes involved in the document
        country_names_involved = list(set(sum(list(entries.Countries.values), start=[])))
    
        # if there are no country names involved, skip entry
        if len(country_names_involved) == 0:
            continue
        
        date = entries.PublicationDate.values[0]
        country_name = country_names_involved[0]  # only one first country name is needed
        
        geojson = ['countries/' + country_dict[country]['iso3'] + '.json' for country in country_names_involved]
        
        lat, lon = country_dict[country_name]['lat'], country_dict[country_name]['lon']
    
        title = 'Human Right Violations in ' + country_name
        text = '<br>'.join(list(entries(.Text.values))

                id INTEGER PRIMARY KEY,
        date TIMESTAMP,
        country_name VARCHAR(150),
        geojson VARCHAR(8000),
        lat DECIMAL(10,7) CHECK (lat >= -90 AND lat <= 90),
        lon DECIMAL(10,7) CHECK (lon >= -180 AND lon <= 180),
        type VARCHAR(200),
        url VARCHAR(500),
        title VARCHAR(500)

        cur.execute('''
INSERT INTO human(id, ;
        ''', {
            'date': 
        }

        connection.commit()
    except:
        connection.rollback()